<a href="https://colab.research.google.com/github/mathu3004/Pearl_Path/blob/Preprocessing/NuwaraEliyaRestaurantsPreprocessed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load the uploaded file to inspect its contents
file_path = 'PreNuwaraEliyaRestaurants.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the data to understand its structure
data.head()

,address,addressObj/city,addressObj/country,addressObj/street1,category,description,email,id,latitude,locationString,...,reviewTags/12/text,reviewTags/13/text,reviewTags/14/text,reviewTags/15/text,reviewTags/16/text,reviewTags/17/text,reviewTags/18/text,reviewTags/19/text,reviewTags/20/text,reviewTags/21/text
0,"18 Badulla Road, Nuwara Eliya 22200 Sri Lanka",Nuwara Eliya,Sri Lanka,18 Badulla Road,restaurant,NaN,NaN,27734687,6.956621,"Nuwara Eliya, Central Province",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"158, Kandy Road, Nuwara Eliya 22220 Sri Lanka",Nuwara Eliya,Sri Lanka,"158, Kandy Road",restaurant,NaN,NaN,15113684,6.974880,"Nuwara Eliya, Central Province",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"20 Kandy Road, Nuwara Eliya 22200 Sri Lanka",Nuwara Eliya,Sri Lanka,20 Kandy Road,restaurant,NaN,NaN,8614101,6.974394,"Nuwara Eliya, Central Province",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"405 Kandy Road Bambarakale, Nuwaraeliya, Nuwar...",Nuwara Eliya,Sri Lanka,"405 Kandy Road Bambarakale, Nuwaraeliya",restaurant,Located on the second floor. Enjoy the beautif...,info@midkyhotels.com,18057087,6.978090,"Nuwara Eliya, Central Province",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Nuwaraeliya Road, Talawakele 22100 Sri Lanka",Talawakele,Sri Lanka,Nuwaraeliya Road,restaurant,NaN,NaN,21337380,6.933452,"Talawakele, Central Province",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# List of column groups to consolidate
columns_to_merge = {
    'dishes': [col for col in data.columns if 'dishes' in col.lower()],
    'features': [col for col in data.columns if 'features' in col.lower()],
    'mealTypes': [col for col in data.columns if 'mealtypes' in col.lower()],
    'reviewTags/Text': [col for col in data.columns if 'reviewtags' in col.lower() and 'text' in col.lower()]
}

# Create consolidated columns
for new_col, cols_to_merge in columns_to_merge.items():
    if cols_to_merge:
        data[new_col] = data[cols_to_merge].apply(lambda row: ', '.join(row.dropna().astype(str)), axis=1)

# Drop the original individual columns if desired
columns_to_drop = [col for cols in columns_to_merge.values() for col in cols]
data = data.drop(columns=columns_to_drop)

# Save the cleaned data to a new file
output_path = 'PreNuwaraEliya_Restaurants_Cleaned.csv'
data.to_csv(output_path, index=False)

output_path

'PreNuwaraEliya_Restaurants_Cleaned.csv'

In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer

# Load the cleaned dataset
cleaned_file_path = 'PreNuwaraEliya_Restaurants_Cleaned.csv'
data = pd.read_csv(cleaned_file_path)

# 1. Remove duplicate rows
data = data.drop_duplicates()

# 2. Handle missing values
# Replace placeholders for unknown values with NaN
unknown_placeholders = ['Unknown', 'N/A', 'NaN', '-', '']
data.replace(unknown_placeholders, pd.NA, inplace=True)

# Identify numerical and categorical columns
num_cols = data.select_dtypes(include=['float64', 'int64']).columns
cat_cols = data.select_dtypes(include=['object']).columns

# Filter out numerical columns with all missing values
valid_num_cols = [col for col in num_cols if data[col].notnull().any()]

# 3. Impute missing values
# Impute numerical columns
if valid_num_cols:
    num_imputer = SimpleImputer(strategy='mean')
    data[valid_num_cols] = num_imputer.fit_transform(data[valid_num_cols])

# Impute categorical columns
if not cat_cols.empty:
    cat_imputer = SimpleImputer(strategy='most_frequent')
    data[cat_cols] = cat_imputer.fit_transform(data[cat_cols])

# Save the preprocessed dataset to a new file
output_path_preprocessed = 'PreNuwaraEliya_Restaurants_Preprocessed.csv'
data.to_csv(output_path_preprocessed, index=False)

# Display the first few rows of the preprocessed data
data.head()


,address,addressObj/city,addressObj/country,addressObj/street1,category,description,email,id,latitude,locationString,...,website,cuisines,dietaryRestrictions,establishmentTypes/0,openHours,closeHours,openNowText,features,mealTypes,reviewTags/Text
0,"18 Badulla Road, Nuwara Eliya 22200 Sri Lanka",Nuwara Eliya,Sri Lanka,18 Badulla Road,restaurant,"""MEMORABLE DINING EXPERIENCES IN NUWARA ELIYA""...",reservations@grandhotel.lk,27734687.0,6.956621,"Nuwara Eliya, Central Province",...,http://thegrandhotelnuwaraeliya.com,"Asian, Sri Lankan",Vegetarian friendly,Restaurants,10:00,22:00,Open Now,"Takeout, Seating, Table Service","Lunch, Dinner","baji, sandwiches, cake, pizza, grand hotel, be..."
1,"158, Kandy Road, Nuwara Eliya 22220 Sri Lanka",Nuwara Eliya,Sri Lanka,"158, Kandy Road",restaurant,"""MEMORABLE DINING EXPERIENCES IN NUWARA ELIYA""...",reservations@grandhotel.lk,15113684.0,6.974880,"Nuwara Eliya, Central Province",...,http://www.pizzahut.lk,"Italian, Pizza, Fast Food",Vegetarian friendly,Restaurants,7:00,22:00,Open Now,"Delivery, Takeout","Breakfast, Lunch, Dinner","baji, sandwiches, cake, pizza, grand hotel, be..."
2,"20 Kandy Road, Nuwara Eliya 22200 Sri Lanka",Nuwara Eliya,Sri Lanka,20 Kandy Road,restaurant,"""MEMORABLE DINING EXPERIENCES IN NUWARA ELIYA""...",reservations@grandhotel.lk,8614101.0,6.974394,"Nuwara Eliya, Central Province",...,http://www.facebook.com/The-Pub-407674586680194/,"Bar, Pub",Vegetarian friendly,Restaurants,7:00,22:00,Open Now,"Takeout, Seating, Television, Serves Alcohol, ...","Lunch, Dinner","fried chicken, grab a quick meal, nice place t..."
3,"405 Kandy Road Bambarakale, Nuwaraeliya, Nuwar...",Nuwara Eliya,Sri Lanka,"405 Kandy Road Bambarakale, Nuwaraeliya",restaurant,Located on the second floor. Enjoy the beautif...,info@midkyhotels.com,18057087.0,6.978090,"Nuwara Eliya, Central Province",...,http://www.midkyhotels.com,"Italian, Chinese, Indian, Asian, Sri Lankan, A...",Vegetarian friendly,Restaurants,11:00,21:00,Open Now,"Takeout, Reservations, Outdoor Seating, Seatin...","Breakfast, Lunch, Dinner","baji, sandwiches, cake, pizza, grand hotel, be..."
4,"Nuwaraeliya Road, Talawakele 22100 Sri Lanka",Talawakele,Sri Lanka,Nuwaraeliya Road,restaurant,"""MEMORABLE DINING EXPERIENCES IN NUWARA ELIYA""...",reservations@grandhotel.lk,21337380.0,6.933452,"Talawakele, Central Province",...,http://thegrandhotelnuwaraeliya.com,"Asian, Sri Lankan",Vegetarian friendly,Restaurants,7:00,22:00,Open Now,"Takeout, Seating, Table Service","Breakfast, Lunch, Dinner","baji, sandwiches, cake, pizza, grand hotel, be..."
